<a href="https://colab.research.google.com/github/Minahil-official/Quater-2/blob/main/Project_03_LangChain_Function_Tool_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *STREAMLIT APP*  
### https://minahil-official-gemini-tool-calling-tool-calling-oiidag.streamlit.app/

## **Creating Environment**

In [1]:
!pip install langchain google-generativeai langchain_google_genai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 110.7 kB/s eta 0:00:00


In [2]:
!pip install python-dotenv -q

In [3]:
!pip install googlesearch-python

## **Key Cofiguration**

In [4]:
from google.colab import userdata
GOOGLE_API_KEY= userdata.get('video_process')

## **Creating Tools**

In [5]:
from langchain_core.tools import tool
@tool
def calculator(expression: str) -> str:
    """
    Evaluates a mathematical expression and returns the result.

    Parameters:
    - expression (str): The mathematical expression to evaluate.

    Returns:
    - str: The result of the evaluation or an error message.
    """
    import math

    # Define a safe dictionary of allowed methods and constants
    allowed_names = {name: getattr(math, name) for name in dir(math) if not name.startswith("__")}
    allowed_names.update({"abs": abs, "round": round})

    try:
        # Use eval with restricted globals for safety
        result = eval(expression, {"__builtins__": None}, allowed_names)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: Invalid expression. Details: {e}"


In [6]:

import requests
news_api_key = "fce15302b99840cca6de1ce31894f4b1"
@tool
def fetch_latest_news(query: str = "latest", language: str = "en", page_size: int = 5):
    """
    Fetches the latest news articles based on a query.

    Parameters:
    - query (str): The keyword for the news search (default is 'latest').
    - language (str): Language for the news (default is 'en').
    - page_size (int): Number of news articles to fetch (default is 5).

    Returns:
    - list: A list of dictionaries with news headlines, descriptions, and URLs.
    """
    base_url = "https://newsapi.org/v2/everything"
    params = {
        "q": query,
        "language": language,
        "pageSize": page_size,
        "apiKey": news_api_key,
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an error for bad responses (4xx, 5xx)
        articles = response.json().get("articles", [])

        news = []
        for article in articles:
            news.append({
                "title": article["title"],
                "description": article["description"],
                "url": article["url"],
            })

        return news
    except Exception as e:
        return {"error": str(e)}


In [7]:
@tool
def fetch_weather(city: str) -> str:
    """
    Fetches the current weather for a given city.

    Parameters:
    - city (str): Name of the city to fetch weather for.

    Returns:
    - str: Weather details or error message.
    """
    api_key = "a7c2649f3fbadc686b85f80b09087df0"  # Replace with your OpenWeatherMap API key
    base_url = "https://api.openweathermap.org/data/2.5/weather"

    params = {
        "q": city,
        "appid": api_key,
        "units": "metric"  # Use "imperial" for Fahrenheit
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise HTTPError for bad responses
        data = response.json()

        # Check if the API returned an error
        if data.get("cod") != 200:
            return f"Error: {data.get('message', 'Unknown error occurred.')}"

        # Extract weather details
        weather = data["weather"][0]["description"]
        temperature = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        city_name = data["name"]

        return (
            f"Weather in {city_name}:\n"
            f"- Description: {weather}\n"
            f"- Temperature: {temperature}°C\n"
            f"- Humidity: {humidity}%"
        )
    except requests.exceptions.RequestException as e:
        return f"Error: Unable to fetch weather data. Details: {e}"
    except KeyError:
        return "Error: Unable to parse weather data. Please check the city name."



In [8]:
@tool
def google_search(query: str, num_results: int = 5):
    """
    Performs a Google search and returns a list of search results.

    Parameters:
    - query (str): The search query.
    - num_results (int): The number of results to return (default is 5).

    Returns:
    - list: A list of search result URLs.
    """
    try:
        # Perform the search
        results = []
        for result in search(query, num_results=num_results):
            results.append(result)

        return results

    except Exception as e:
        return {"error": str(e)}


## **Adding Tools**

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp" , api_key=GOOGLE_API_KEY)

from langchain.agents import initialize_agent, AgentType

# Define the tools list using the Tool objects
# and the calculator function decorated with @tool
tools = [calculator, fetch_latest_news, fetch_weather,google_search]

agent = initialize_agent(tools, llm , agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION )


<ipython-input-9-8529c2845c9e>:11: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm , agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION )


## **Getting Response**

In [10]:
respone =  agent.invoke({"input":"give google search about pakistan"})
respone["output"]

"Pakistan is a country in South Asia. It is the fifth-most populous country in the world with a population exceeding 240 million people. It shares borders with Iran, Afghanistan, China, and India. The capital city is Islamabad, and the largest city is Karachi. Pakistan has a rich history and diverse culture. It gained independence in 1947 and is a parliamentary republic. The official languages are Urdu and English. The country's economy is the 25th largest in the world in terms of purchasing power parity."